In [1]:
from analysis.read_logs import load_benchmark_dir_dict
import polars as pl
import plotly.express as px
from analysis.analysis import summary, plot_ops_per_sec, plot_mem, plot_disk_usage

In [2]:
mixed_small = load_benchmark_dir_dict("/Users/arc/iavl-bench-data/run-mixed-small")

In [3]:
summary(mixed_small, ['iavl-v1', 'iavl-v2-alpha6', 'memiavl'])

name,ops_per_sec,max_mem_gb,max_disk_gb
str,f64,f64,f64
"""iavl-v1""",4742.310918,1.559486,153.0
"""iavl-v2-alpha6""",10644.822227,0.996339,50.0
"""memiavl""",20543.653772,14.627673,26.0


In [4]:
plot_ops_per_sec(mixed_small, ['iavl-v1', 'iavl-v2-alpha6', 'memiavl'])

In [5]:
plot_mem(mixed_small, ['iavl-v1', 'iavl-v2-alpha6', 'memiavl'])

In [6]:
summary(mixed_small, ['iavl-v2-alpha6-evict20', 'iavl-v2-alpha6', 'memiavl'])

name,ops_per_sec,max_mem_gb,max_disk_gb
str,f64,f64,f64
"""iavl-v2-alpha6-evict20""",15925.232166,15.010651,50.0
"""iavl-v2-alpha6""",10644.822227,0.996339,50.0
"""memiavl""",20543.653772,14.627673,26.0


In [7]:
plot_ops_per_sec(mixed_small, ['iavl-v2-alpha6-evict20', 'iavl-v2-alpha6', 'memiavl'])

In [ ]:
mixed_large = load_benchmark_dir_dict("/Users/arc/iavl-bench-data/run-mixed-large")

In [ ]:
summary(mixed_large)

In [ ]:
plot_ops_per_sec(mixed_large, ["memiavl", "iavl-v1", "iavl-v2-alpha6", "memiavl-100-2"])

In [ ]:
plot_mem(mixed_large, ["memiavl", "iavl-v1", "iavl-v2-alpha6", "memiavl-100-2"])

In [ ]:
plot_disk_usage(mixed_large, ["memiavl", "iavl-v1", "iavl-v2-alpha6", "memiavl-100-2"])
